In [1]:
import numpy as np
import syft as sy
import pandas as pd

In [2]:
from time import time

In [3]:
data = pd.read_parquet("/mnt/hdd1/1M_rows_dataset_sample.parquet")

In [4]:
data.head()

,tweet_id,impressions,impressions_ct,bucket,tweet_origin,tweet_date_time,date,time,user_id,user_country,url,publication_title,ad_fontes_bias,ad_fontes_reliability,domain
0,59,9036,1870,0,Japan,2021-05-03 01:38:55,2021-05-03,01:38:55,413396,Germany,https://www.alternet.org/2018/09/michael-moore...,AlterNet,-28.25,10.75,.alternet.org
1,59,9036,1870,0,Japan,2021-05-03 01:38:55,2021-05-03,01:38:55,487138,Canada,https://www.alternet.org/2018/09/michael-moore...,AlterNet,-28.25,10.75,.alternet.org
2,59,9036,1870,0,Japan,2021-05-03 01:38:55,2021-05-03,01:38:55,476426,Canada,https://www.alternet.org/2018/09/michael-moore...,AlterNet,-28.25,10.75,.alternet.org
3,59,9036,1870,0,Japan,2021-05-03 01:38:55,2021-05-03,01:38:55,407094,U.K.,https://www.alternet.org/2018/09/michael-moore...,AlterNet,-28.25,10.75,.alternet.org
4,59,9036,1870,0,Japan,2021-05-03 01:38:55,2021-05-03,01:38:55,683755,Canada,https://www.alternet.org/2018/09/michael-moore...,AlterNet,-28.25,10.75,.alternet.org


In [5]:
data["tweet_id"].unique()

array([59, 63, 62, 64, 51, 75, 60, 81, 53, 71, 50, 57, 70, 82, 78, 69, 52,
       77, 72, 55, 61, 79, 66, 65, 56, 74, 68, 73, 58, 32, 76, 14, 30, 27,
       67, 20, 80, 28, 16, 12,  7, 24,  8,  4, 21, 43,  6, 23, 25, 46, 33,
       29, 31, 35])

In [6]:
users = data["user_id"].to_numpy(dtype=np.int64)[:10]

In [7]:
users.min(), users.max()

(71023, 683755)

In [8]:
tweet_id_data = data["tweet_id"].to_numpy(dtype=np.int64)[:10]

In [9]:
tweet_id_data.shape

(10,)

In [10]:
t0 = time()
tweet_ids = sy.Tensor(tweet_id_data).private(min_val=0, max_val=60, data_subjects=users)
tf = time()
print("Time to annotate: ", tf-t0)

Time to annotate:  0.004360198974609375


In [11]:
bucket_data = data["bucket"].to_numpy(dtype=np.int64)[:10]

In [12]:
tweet_buckets = sy.Tensor(bucket_data).private(min_val=0, max_val=1, data_subjects=users)

In [13]:
tweet_user_id = sy.Tensor(users).private(min_val=0, max_val=700_000, data_subjects=users)

In [14]:
domain = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into adp... done!


In [15]:
domain.load_dataset(
    assets={
        "tweet_id": tweet_ids,
        "buckets": tweet_buckets,
        "user_id": tweet_user_id
    },
    name="Twitter Dataset",
    description="Synthetic Twitter Dataset"
)

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/e/PycharmProjects/PySyft/notebooks/Experimental/Ishan


Loading dataset... uploading...🚀                                                                                                                                             

Uploading `user_id`: 100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 151.84it/s]


Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


In [16]:
domain.create_user(name="DataScientist", email="ds@ds.ai", password="mlaids", budget=1000)

User created successfully!


{'name': 'DataScientist',
 'email': 'ds@ds.ai',
 'password': 'mlaids',
 'url': 'localhost'}

In [17]:
ds_node = sy.login(email="ds@ds.ai", password="mlaids", port=8081)

Connecting to localhost... done! 	 Logging into adp... done!


In [18]:
ds_node.datasets

Idx,Name,Description,Assets,Id
[0],Twitter Dataset,Synthetic Twitter Dataset,"[""tweet_id""] -> [""buckets""] -> [""user_id""] ->",2894e563-d730-4e71-bbad-17f4011f6e1d


In [21]:
tweet_ids = ds_node.datasets[0]["tweet_id"]
buckets = ds_node.datasets[0]["buckets"]
user_id = ds_node.datasets[0]["user_id"]

In [22]:
chron_impression_idx = buckets == 0

In [23]:
chron_impression_idx.exists

True

In [24]:
chron_impressions = chron_impression_idx.sum()

In [25]:
chron_impressions.exists

True

In [26]:
algo_impression_idx = buckets == 1

In [27]:
algo_impression_idx.exists

True

In [28]:
algo_impressions = algo_impression_idx.sum()

In [29]:
algo_impressions.exists

True

In [30]:
amp_ratio = algo_impressions/chron_impressions

/home/e/PycharmProjects/PySyft/packages/syft/src/syft/core/tensor/lazy_repeat_array.py:216: RuntimeWarning: invalid value encountered in true_divide
  return self.__class__(data=self.data / other.data, shape=self.shape)


In [37]:
amp_ratio.publish(sigma=1)

<AnyPointer -> adp:fe94900fee2c49d2b4d9d8661c9761af>

In [33]:
ds_node.privacy_budget

1000.0

In [34]:
x

<AnyPointer -> adp:9b3ef79d371641cba0bfbb8cfc60f818>

In [35]:
ratio = x.get()

In [36]:
ratio

1.1602989161731234